In [ ]:
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import gradio as gr
import requests
import io
import base64
import datetime

######################################
# Azure 관련 전역 변수
######################################
 
OPENAI_ENDPOINT = "https://fimtrus-ai-project-resource.cognitiveservices.azure.com/"    
OPENAI_API_KEY = "9VEZmMXEZd4aWzFIEVBKCse1NEa3en2LrD51oEyXbFc41XDbcP2VJQQJ99BFACYeBjFXJ3w3AAAAACOGydeD"
DEPLOYMENT_NAME = "fimtrus-gpt-41"
 
SPEECH_ENDPOINT = "https://eastus.tts.speech.microsoft.com/cognitiveservices/v1"
SPEECH_API_KEY = "45SFNywqabNccYBIp1fGDsJKJYe5N1pfPZLrSlLGY1ebDzyPLT75JQQJ99BFACYeBjFXJ3w3AAAYACOGWzIo"


weights_path = "yolo/yolov3.weights"
config_path = "yolo/yolov3.cfg"
names_path = "yolo/coco.names"


with open(names_path, 'r', encoding='utf-8') as file:
    label_list = file.read().strip().split("\n")

net = cv2.dnn.readNet(weights_path, config_path)


def request_gpt(image_array):

    # PIL 형태의 이미지
    image = Image.fromarray(image_array)

    byte_image = io.BytesIO()
    image.save(byte_image, format="png")
    base64_image = base64.b64encode(byte_image.getvalue()).decode('utf-8')

    # TODO: buffered_io to base64
    endpoint = "{}openai/deployments/{}/chat/completions?api-version=2025-01-01-preview".format(OPENAI_ENDPOINT, DEPLOYMENT_NAME)
 
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(OPENAI_API_KEY)
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        너는 사진속에서 감지된 물체를 분석하는 AI 봇이야.
                        무조건 분석 결과를 한국어로 답변해줘.
                        """
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        너는 물체를 감지하는 YOLO 모델이야.
                        이 사진에서 감지된 물체에 대해서 감지확률과 자세한 설명을 붙여줘.
                        반드시 감지된 물체, 바운딩 박스 안에 있는 물체에 대해서만 설명해줘.
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            },
        ],
        "temperature": 0.7,
        "top_p": 0.9,
        "max_tokens": 16000
    }

    response = requests.post(endpoint, headers=headers, json=body)

    if response.status_code != 200:
        return None
    
    response_json = response.json()
    content = response_json['choices'][0]['message']['content']

    return content


def request_tts(text):
    endpoint = SPEECH_ENDPOINT
    headers = {
        "Ocp-Apim-Subscription-Key": SPEECH_API_KEY,
        "Content-Type": "application/ssml+xml",
        "X-Microsoft-OutputFormat": "riff-8khz-16bit-mono-pcm"
    }
    body = f"""
        <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xmlns:mstts="http://wwww.w3.org/2001/mstts" xml:lang="ko-KR">
        <voice name="ko-KR-SeoHyeonNeural">
            {text}
    </voice>
    </speak>

    """

    response = requests.post(endpoint, headers=headers, data=body)
    
    if response.status_code != 200:
        return None

    file_name = "tts_result_{}.wav".format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

    with open(file_name, "wb") as audio_file:
        audio_file.write(response.content)

    return file_name


def random_color():
    # 랜덤한 RGB 색상 튜플 반환
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 


def get_font():
    # OS별로 적절한 폰트 객체 반환 (한글 지원)
    from PIL import ImageFont
    import platform
    
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux      
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 폰트 파일이 없을 경우 기본 폰트 사용
        return ImageFont.load_default(size=font_size)
    

def detect_object(image_array):

    image = Image.fromarray(image_array.copy())
    draw = ImageDraw.Draw(image)
    font = get_font()

    height, width = image_array.shape[:2]
    blob = cv2.dnn.blobFromImage(image_array, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_name_list = net.getLayerNames()
    out_layer_list = net.getUnconnectedOutLayersNames()
    detection_list = net.forward(out_layer_list)

    bounding_box_list = list()
    confidence_list = list()
    label_index_list = list()
    
    for prediction_list in detection_list:
        color = random_color()
        #yolo82, yolo94, yolo 106
        for prediction in prediction_list:
            score_list = prediction[5:]
            label_index = np.argmax(score_list)
            confidence = score_list[label_index]

            if confidence > 0:
                bounding_box = prediction[:4] * np.array([width, height, width, height])
                center_x, center_y, w, h = bounding_box.astype('int')
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                # print(x, y, w, h)

                bounding_box_list.append([x, y, w, h])
                confidence_list.append(confidence)
                label_index_list.append(label_index)

                # draw.rectangle([(x, y), (x + w, y + h)], outline='red', width=2)

    extracted_index_list = cv2.dnn.NMSBoxes(bounding_box_list, confidence_list, 0.5, 0.4)

    for extracted_index in extracted_index_list:

        x, y, w, h = bounding_box_list[extracted_index]
        confidence = confidence_list[extracted_index]
        label_index = label_index_list[extracted_index]
        label_text = label_list[label_index]

        draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=2)
        draw.text([x + 5, y + 5], "{}({:.2f}%)".format(label_text, confidence * 100), fill=color, font=font)

    return image


with gr.Blocks(theme=gr.themes.Soft()) as demo:

    def stream_webcam(image):
        result_image = detect_object(image)
        return result_image
    

    def click_capture(image):
        if image is None:
            raise gr.Error("감지된 이미지가 없습니다.", duration=3)
        return image 
    

    def click_send_gpt(image_array, histories):
        content = request_gpt(image_array)
        print("CLICK_SEND_GPT", content)
        now = datetime.datetime.now()
        label_text = now.strftime("%Y-%m-%d_%H:%M:%S")
        histories.append({"role": "user", "content": gr.Image(label=label_text, value=image_array)})
        histories.append({"role": "assistant", "content":content})
        # 오른쪽엔 사진, 왼쪽엔 컨텐츠
        return histories
    

    def change_chatbot(histories):
        content = histories[-1]['content']
        file_path = request_tts(content)
        return file_path

    with gr.Row():
        webcam_image = gr.Image(label="실시간 화면", sources="webcam", width=480, height=270, mirror_webcam=False)
        output_image= gr.Image(label="검출 화면", type="pil", width=480, height=270)
        output_capture_image = gr.Image(label="이상 징후 캡쳐 화면", interactive=False, width=480, height=270)

    with gr.Row():
        capture_button = gr.Button("이상 징후 발생")
        send_gpt_button = gr.Button("분석")

    chatbot = gr.Chatbot(label="감지 내용", type="messages")
    chatbot_audio = gr.Audio(label="감지 내용", interactive=False, autoplay=True)

    webcam_image.stream(stream_webcam, inputs=[webcam_image], outputs=[output_image])
    capture_button.click(click_capture, inputs=[output_image], outputs=[output_capture_image])
    send_gpt_button.click(click_send_gpt, inputs=[output_capture_image, chatbot], outputs=[chatbot])

    chatbot.change(change_chatbot, inputs=[chatbot], outputs=[chatbot_audio])

demo.launch()

# test_image = cv2.imread("C:/Users/USER/Downloads/ImageTaggingSample1-fd324157.jpg")
# test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
# request_gpt(test_image)

# detect_object(test_image)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\components\image.py:134: UserWarning: The `mirror_webcam` parameter is deprecated. Please use the `webcam_options` parameter with a `gr.WebcamOptions` instance instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


CLICK_SEND_GPT 아래는 사진 속에서 바운딩 박스로 감지된 물체와 감지 확률, 그리고 자세한 설명입니다.

1. 왼쪽 위 (감지확률: 58.04%)
   - 감지된 물체: 고양이(새끼 고양이)
   - 설명: 귀여운 새끼 고양이가 앞을 바라보고 앉아 있는 모습입니다. 흰색과 회색 줄무늬가 있으며, 똑바로 앉아 카메라를 응시하고 있습니다.

2. 가운데 위 (감지확률: 81.26%)
   - 감지된 물체: 고양이
   - 설명: 갈색 고양이가 뒤돌아 앉아 바깥 풍경을 바라보고 있습니다. 뒷모습이 주로 보이며, 자연 환경 속에 있는 모습입니다.

3. 왼쪽 아래 (감지확률: 93.66%)
   - 감지된 물체: 고양이
   - 설명: 회색 털의 성묘 고양이가 정면을 바라보고 있습니다. 얼굴이 크게 클로즈업되어 있으며, 눈이 매우 또렷하게 보입니다.

4. 오른쪽 아래 (감지확률: 51.41%)
   - 감지된 물체: 고양이
   - 설명: 주황색 줄무늬 고양이가 바닥에 길게 누워서 쉬고 있습니다. 앞발을 쭉 뻗고, 편안한 자세를 취하고 있습니다.

이상 네 마리 고양이가 바운딩 박스 안에서 감지되었습니다.
CLICK_SEND_GPT 이 사진에서 YOLO 모델이 감지한 물체는 다음과 같습니다:

1. 고양이 (cat) - 감지 확률: 96.77%  
   - 위치: 사진 왼쪽 중단, 바운딩 박스 안에 있는 갈색과 흰색 털을 가진 어린 고양이입니다. 귀가 크고 눈이 동그랗게 떠 있으며, 몸집이 작고 귀여운 새끼 고양이로 보입니다.

2. 고양이 (cat) - 감지 확률: 99.07%  
   - 위치: 사진 중앙 중단, 바운딩 박스 안에 있는 회색과 흰색 털을 가진 어린 고양이입니다. 털이 풍성하고 얼굴이 둥글며, 호기심 많은 표정을 짓고 있습니다.

3. 고양이 (cat) - 감지 확률: 89.56%  
   - 위치: 사진 왼쪽 하단, 바운딩 박스 안에 있는 회색 줄무늬의 새끼 고양이입니다. 귀가 크고 눈이 크며, 앞발을 모으고 앉아 있는 모습입니다.

